# Jointformer Examples

This notebook shows how to use Jointformer, as a SMILES encoder, generator and how to train the model with default Trainer. 

In [15]:
# Imports

import os
import torch

from jointformer.configs.tokenizer import TokenizerConfig
from jointformer.configs.model import ModelConfig

from jointformer.utils.tokenizers.auto import AutoTokenizer
from jointformer.models.auto import AutoModel


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Set working directory of the project

REPOSITORY_DIR = '/home/adamizdebski/projects/jointformer'
os.chdir(REPOSITORY_DIR)

In [17]:
# Configs

DATA_DIR = '/home/adamizdebski/files/data'
OUTPUT_DIR = '/home/adamizdebski/files/jointformer/results/pretrain'

PATH_TO_TOKENIZER_CONFIG = '/home/adamizdebski/projects/jointformer/configs/tokenizers/smiles_separate_task_token'
PATH_TO_MODEL_CONFIG = '/home/adamizdebski/projects/jointformer/configs/models/jointformer_separate_task_token'
PRETRAINED_JOINTFORMER_FILENAME = 'ckpt.pt'


In [18]:
# Init params and tokenizer


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

tokenizer_config = TokenizerConfig.from_config_file(PATH_TO_TOKENIZER_CONFIG)
tokenizer = AutoTokenizer.from_config(tokenizer_config)

In [19]:
# Init Jointformer

model_config = ModelConfig.from_config_file(PATH_TO_MODEL_CONFIG)
model = AutoModel.from_config(model_config)
model.load_pretrained(PRETRAINED_JOINTFORMER_FILENAME)

## Jointformer as a SMILES Encoder

In [20]:
# Init Smiles Encoder as a wrapper for molecule encoding

encoding_batch_size = 16
smiles_encoder = model.to_smiles_encoder(tokenizer=tokenizer, device=device, batch_size=encoding_batch_size)

In [21]:
# Encode smiles

smiles = [
    "COC(=O)c1cc(C(=CCCCC(=O)SC)c2cc(Cl)c(OC)c(C(=O)OC)c2)cc(Cl)c1OC",
    "Cc1cc(COc2ccc(NC(=O)C3CN(C)CCC3C(=O)NO)cc2)c2ccccc2n1",
    "CCOC(=O)c1ccc(O)c(-n2cc3c(c2-c2ccccc2)c(=O)n(C)c(=O)n3C)c1",
    "COc1cc(OC)c2c(-c3cccc(-c4ccc(C#N)cc4)c3)cc(=O)oc2c1",
    "COc1cc2[nH]c(C(=O)Nc3ccc(F)cc3)cc2c(OC)c1OC",
    "COc1cc(F)cc(-c2ccc(C(CC(=O)O)NC(=O)C3CCCN3S(=O)(=O)c3cc(Cl)cc(Cl)c3)cc2)c1",
    "COc1cc(-c2cc(OC)c(-n3c(=O)ccc4cc(S(=O)(=O)Nc5cccnn5)ccc43)cc2F)ccc1Cl",
    "CCCCC1(C)CC(CO)C(CCCC)(OC)OO1"
]

encoding = smiles_encoder.encode(smiles)  # setting the device and model.eval() under the hood

Encoding samples: 100%|██████████| 1/1 [00:00<00:00,  4.05it/s]


## Jointformer as a SMILES generator

In [22]:
batch_size = 8
temperature = 0.8
top_k = 10

generator = model.to_guacamole_generator(tokenizer=tokenizer, batch_size=batch_size, temperature=temperature, top_k=top_k, device=device)

In [23]:
generator.generate(number_samples=16)

Generating samples: 100%|██████████| 2/2 [00:01<00:00,  1.02it/s]


['COC1=CC=C(C(=O)C=CC2=CC=C(C)S2)C=C1OCC1=CC=C([N+](=O)[O-])C=C1',
 'C1(C)=CC=C(C2N(C(=O)C)N=C3C2CCC2(C)C(C(=O)OCC)CC(O)C(=O)C2=C3)O1',
 'C1=CC=C(C(NCC2=NC3=C(C=CC=C3)N2)C)C=C1OC(F)(F)F',
 'N1=C(C)C2=CC=CC=C2N2C1=NC(CN1CCCC1)=C2',
 'CCCN(CCC)C(=O)C1=CC(C(C)NC2=CC(F)=CC(F)=C2)=C2C=CC=CN2C1=O',
 'C1=C(C(=O)C2=C(C)NC(C(OCC)=O)=C2C)C=CC=C1',
 'N1=C(NC2=CC=C(OC3=CC=CC=C3)C=C2)C(S(=O)(=O)C2=CC=CC=C2)=NC2=C1C=CC=C2',
 'CC(C)C1=C(C(=O)O)N2C(=O)C(C(C)O)C2S1(=O)=O',
 'C1=CC=C(N2C(C)=C(C(=O)NC3=C(F)C=CC=C3)N=N2)C=C1C',
 'OC1=CC=C(C(=O)C=CC2=CC=C(O)C=C2)C=C1',
 'C1=CC=CC=C1C(NC(=O)NC1=C(C(=O)OC)SC=C1)=O',
 'O=C1C(C(N2CCN(C(=O)C3=CC=CO3)CC2)C2=CC=C(C)C=C2)=C(O)N=C2C=C(OC)C(Cl)=CC2=N1',
 'C1(NC(C2=C(N)C3=C(N=C(CC(C)C)N3)N=C2)=O)=CC=C(F)C(F)=C1',
 'C(CN1C(=O)CCC1)CNC(C1=CC=CC=C1OCC)=O',
 'C1=CC=CC=C1CCN1C(=O)NC(=O)C(N(C(C2=C(C)ON=C2C2=C(F)C=CC=C2Cl)=O)CCCC)=C1N',
 'N1=CN2C(=C1)CN(C(C1=CC=C(F)C=C1Cl)=O)CC2']